# PDF parser

Firstly, looks like the pdf to parse has a text level to it, if this is the case then extracting the text should be no real conflict and any text extraction library may do the job:

## Text from file

In [67]:
from PyPDF2 import PdfReader
reader = PdfReader("../CONTRATO_AP000000718.pdf")
  
for i in range(len(reader.pages)):
    page = reader.pages[i]
    text = page.extract_text()
    print(text)
    print(i)


0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29


notice, each page reads empty, lets test another method:

## pdfminer

In [72]:
from pdfminer import high_level

extracted_text = high_level.extract_text("../CONTRATO_AP000000718.pdf", "", range(0,29))
print(extracted_text)

That didn't work either. Let's try a few more:

In [74]:
import fitz

pdf = fitz.open(../"CONTRATO_AP000000718.pdf")

for page in pdf:
    text = page.get_text("text")
    print(text)

In [75]:
import textract

text = textract.process('../CONTRATO_AP000000718.pdf', encoding='latin')
print(text)

b'\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c'


Notice how textract only outputs the end of page literal representations. This shows that the text in the file is empty. This leads to only 1 option, osing OCR.

# OCR

## Propietary OCR

Creating an OCR from pdf requires 2 steps, a function to transform each page into an image and later the OCR function that reads the image.

In [4]:
import pytesseract
from pdf2image import convert_from_path
import PyPDF2

In [11]:
image = convert_from_path("../CONTRATO_AP000000718.pdf", first_page=0, last_page=0 + 1)
text = pytesseract.image_to_string(image[0], lang="spa")
print(text)

CONTRATO MAESTRO NUMERO AP000000718 DE ARRENDAMIENTO DE BIENES
MUEBLES (EN LO SUCESIVO DENOMINADO EL “ARRENDAMIENTO MAESTRO”) QUE
CELEBRAN POR UNA PARTE AB2C LEASING DE MEXICO, SOCIEDAD ANÓNIMA
PROMOTORA DE INVERSIÓN DE CAPITAL VARIABLE. (EL “ARRENDADOR”,
REPRESENTADA POR MARÍA ISABEL BOLIO' MONTERO Y PABLO ENRIQUE ROMERO
GONZÁLEZ , POR OTRA PARTE LA EMPRESA; CRANE SUPPLIES SERVICES S.A. de
C.V. REPRESENTADA POR ÓSCAR ALBERTO ISLAS MENDOZA ( “EL ARRENDATARIO”
), POR OTRA PARTE: EN LO PERSONAL Y POR SU PROPIO DERECHO, OSCAR
ALBERTO ISLAS MENDOZA (COMO “EL OBLIGADO SOLIDARIO”), POR ULTIMO EN LO
PERSONAL Y POR SU PROPIO DERECHO OSCAR ALBERTO ISLAS MENDOZA, COMO
(EL DEPOSITARIO”) DE ACUERDO CON LAS SIGUIENTES DECLARACIONES Y
CLAUSULAS.

DECLARACIONES
E. El Arrendador declara, representa y garantiza que:

a. Es una Sociedad Anónima Promotora de Inversión de Capital Variable debidamente
coristituida. bajo el nombre de Boston Leasing México, S.A. de C.Y., de conformidad a las
léyes de lós Est

like so, and then all can be wrapped in a function for the whole file:

In [12]:
def extract_text_from_pdf_tesseract(pdf_path):
    # Read the PDF file
    pdf_reader = PyPDF2.PdfReader(open(pdf_path, 'rb'))

    # Initialize an empty list to store extracted text from each page
    extracted_text_list = []

    # Loop through each page of the PDF
    for page_num in range(len(pdf_reader.pages)):
        # Convert the current page to an image using pdf2image
        images = convert_from_path(pdf_path, first_page=page_num, last_page=page_num + 1)

        # Extract text from the image using pytesseract
        text = pytesseract.image_to_string(images[0], lang="spa")
        extracted_text_list.append(text)

    return extracted_text_list

pdf_path = "../CONTRATO_AP000000718.pdf"
extracted_text_list = extract_text_from_pdf_tesseract(pdf_path)

for page_num, text in enumerate(extracted_text_list, start=1):
    print(f"Page {page_num}:\n{text}\n")

Page 1:
CONTRATO MAESTRO NUMERO AP000000718 DE ARRENDAMIENTO DE BIENES
MUEBLES (EN LO SUCESIVO DENOMINADO EL “ARRENDAMIENTO MAESTRO”) QUE
CELEBRAN POR UNA PARTE AB2C LEASING DE MEXICO, SOCIEDAD ANÓNIMA
PROMOTORA DE INVERSIÓN DE CAPITAL VARIABLE. (EL “ARRENDADOR”,
REPRESENTADA POR MARÍA ISABEL BOLIO' MONTERO Y PABLO ENRIQUE ROMERO
GONZÁLEZ , POR OTRA PARTE LA EMPRESA; CRANE SUPPLIES SERVICES S.A. de
C.V. REPRESENTADA POR ÓSCAR ALBERTO ISLAS MENDOZA ( “EL ARRENDATARIO”
), POR OTRA PARTE: EN LO PERSONAL Y POR SU PROPIO DERECHO, OSCAR
ALBERTO ISLAS MENDOZA (COMO “EL OBLIGADO SOLIDARIO”), POR ULTIMO EN LO
PERSONAL Y POR SU PROPIO DERECHO OSCAR ALBERTO ISLAS MENDOZA, COMO
(EL DEPOSITARIO”) DE ACUERDO CON LAS SIGUIENTES DECLARACIONES Y
CLAUSULAS.

DECLARACIONES
E. El Arrendador declara, representa y garantiza que:

a. Es una Sociedad Anónima Promotora de Inversión de Capital Variable debidamente
coristituida. bajo el nombre de Boston Leasing México, S.A. de C.Y., de conformidad a las
léyes de

Then it is enumerated in a list:

In [13]:
paged_text = []
for page_num, text in enumerate(extracted_text_list, start=1):
    curr_page = f'Pagina {page_num} \n' + text
    paged_text.append(curr_page)

In [14]:
paged_text

["Pagina 1 \nCONTRATO MAESTRO NUMERO AP000000718 DE ARRENDAMIENTO DE BIENES\nMUEBLES (EN LO SUCESIVO DENOMINADO EL “ARRENDAMIENTO MAESTRO”) QUE\nCELEBRAN POR UNA PARTE AB2C LEASING DE MEXICO, SOCIEDAD ANÓNIMA\nPROMOTORA DE INVERSIÓN DE CAPITAL VARIABLE. (EL “ARRENDADOR”,\nREPRESENTADA POR MARÍA ISABEL BOLIO' MONTERO Y PABLO ENRIQUE ROMERO\nGONZÁLEZ , POR OTRA PARTE LA EMPRESA; CRANE SUPPLIES SERVICES S.A. de\nC.V. REPRESENTADA POR ÓSCAR ALBERTO ISLAS MENDOZA ( “EL ARRENDATARIO”\n), POR OTRA PARTE: EN LO PERSONAL Y POR SU PROPIO DERECHO, OSCAR\nALBERTO ISLAS MENDOZA (COMO “EL OBLIGADO SOLIDARIO”), POR ULTIMO EN LO\nPERSONAL Y POR SU PROPIO DERECHO OSCAR ALBERTO ISLAS MENDOZA, COMO\n(EL DEPOSITARIO”) DE ACUERDO CON LAS SIGUIENTES DECLARACIONES Y\nCLAUSULAS.\n\nDECLARACIONES\nE. El Arrendador declara, representa y garantiza que:\n\na. Es una Sociedad Anónima Promotora de Inversión de Capital Variable debidamente\ncoristituida. bajo el nombre de Boston Leasing México, S.A. de C.Y., de conf

Notice, that this comes with it's caveats, page 1 and 2 are duplicated, some words have spelling errors and there are some punctuation errors.

## Unstructured OCR

[Unstructured.IO](https://github.com/Unstructured-IO) has an open source library for transforming data. It has the function `partition_pdf` wich is built on top of pytesseract but has finetuned models for OCR. Using the high resolution model strategy results in a better parsed text:

In [16]:
from unstructured.partition.pdf import partition_pdf


# Applies the English and Swedish language pack for ocr. OCR is only applied
# if the text is not available in the PDF.
elements = partition_pdf("CONTRATO_AP000000718.pdf", ocr_languages="spa", strategy="hi_res",)

[nltk_data] Downloading package punkt to /home/isaid/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/isaid/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
unstructured_inference is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with another strategy.
Falling back to partitioning with ocr_only.


In [47]:
parsed_better = []
curr_page = 0
curr_page_text = ''
for line in elements:
    # Separate between pages
    if curr_page != line.metadata.page_number:
        parsed_better.append(str(curr_page_text))
        curr_page = line.metadata.page_number
        curr_page_text = (f'Page {curr_page}\n')
        
    curr_page_text+=f'\n{line.text}'

parsed_better

['',
 "Page 1\n\nCONTRATO MAESTRO NUMERO AP000000718 DE ARRENDAMIENTO DE BIENES MUEBLES (EN LO SUCESIVO DENOMINADO EL “ARRENDAMIENTO MAESTRO”) QUE CELEBRAN POR UNA PARTE AB2C LEASING DE MEXICO, SOCIEDAD ANÓNIMA PROMOTORA DE INVERSIÓN DE CAPITAL VARIABLE. (EL “ARRENDADOR”, REPRESENTADA POR MARÍA ISABEL BOLIO' MONTERO Y PABLO ENRIQUE ROMERO GONZÁLEZ , POR OTRA PARTE LA EMPRESA; CRANE SUPPLIES SERVICES S.A. de C.V. REPRESENTADA POR ÓSCAR ALBERTO ISLAS MENDOZA ( “EL ARRENDATARIO” ), POR OTRA PARTE: EN LO PERSONAL Y POR SU PROPIO DERECHO, OSCAR ALBERTO ISLAS MENDOZA (COMO “EL OBLIGADO SOLIDARIO”), POR ULTIMO EN LO PERSONAL Y POR SU PROPIO DERECHO OSCAR ALBERTO ISLAS MENDOZA, COMO (EL DEPOSITARIO”) DE ACUERDO CON LAS SIGUIENTES DECLARACIONES Y CLAUSULAS.\nDECLARACIONES E. El Arrendador declara, representa y garantiza que:\na. Es una Sociedad Anónima Promotora de Inversión de Capital Variable debidamente coristituida. bajo el nombre de Boston Leasing México, S.A. de C.Y., de conformidad a las

In [48]:
out_string = '\n\n'.join(parsed_better)
out_string

'\n\nPage 1\n\nCONTRATO MAESTRO NUMERO AP000000718 DE ARRENDAMIENTO DE BIENES MUEBLES (EN LO SUCESIVO DENOMINADO EL “ARRENDAMIENTO MAESTRO”) QUE CELEBRAN POR UNA PARTE AB2C LEASING DE MEXICO, SOCIEDAD ANÓNIMA PROMOTORA DE INVERSIÓN DE CAPITAL VARIABLE. (EL “ARRENDADOR”, REPRESENTADA POR MARÍA ISABEL BOLIO\' MONTERO Y PABLO ENRIQUE ROMERO GONZÁLEZ , POR OTRA PARTE LA EMPRESA; CRANE SUPPLIES SERVICES S.A. de C.V. REPRESENTADA POR ÓSCAR ALBERTO ISLAS MENDOZA ( “EL ARRENDATARIO” ), POR OTRA PARTE: EN LO PERSONAL Y POR SU PROPIO DERECHO, OSCAR ALBERTO ISLAS MENDOZA (COMO “EL OBLIGADO SOLIDARIO”), POR ULTIMO EN LO PERSONAL Y POR SU PROPIO DERECHO OSCAR ALBERTO ISLAS MENDOZA, COMO (EL DEPOSITARIO”) DE ACUERDO CON LAS SIGUIENTES DECLARACIONES Y CLAUSULAS.\nDECLARACIONES E. El Arrendador declara, representa y garantiza que:\na. Es una Sociedad Anónima Promotora de Inversión de Capital Variable debidamente coristituida. bajo el nombre de Boston Leasing México, S.A. de C.Y., de conformidad a las 

And finally, it's possible to save the entire parsed text to pass as context to the LLM Model:

In [49]:
with open('parsed_hi-res.txt','w') as to_save:
    to_save.write(out_string)
    to_save.close()

These functionalities will be added to the arkaham api, defaulting the parsing with the high resolution implementation and the option for a secure and faster implementation with the propietary one.